In [ ]:
!pip install bitsandbytes
!pip install jsonlines
!pip install datasets
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl'

In [ ]:
import os

klasor = "/content/drive/MyDrive/Classroom"
print("Klasördeki dosyalar:", os.listdir(klasor))

Klasördeki dosyalar: ['İletişim ve Sunum Teknikleri Bilgisayar Mühendisliği Bölümü', 'hukuk_finetune_temizlenmis (1).jsonl']


In [ ]:
print(data_path)
with open(data_path, "r", encoding="utf-8") as dosya:
    for i, satir in enumerate(dosya):
        print(f"{i+1}. satır: {satir.strip()}")
        if i >= 9:  # İlk 10 satırı yazdır
            break

/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl
1. satır: {"batch": 1, "title": "Batch 1", "prompt": "KISALTMALAR AY: Anayasa BK: Borçlar Kanunu CK: Ceza Kanunu CBK: Cumhurbaşkanlığı Kararnamesi CMUK: Ceza Muhakemeleri Usûlü Kanunu CMK: Ceza Muhakemesi Kanunu f.: Fıkra HUMK: Hukuk Usûlü Muhakemeleri Kanunu HMK: Hukuk Usûlü Muhakemeleri Kanunu İİK: İcra ve İflas Kanunu ÎYUK: İdarî Yargılama Usûlü Kanunu KHK: Kanun hükmünde kararname m.: Madde MK: Medeni Kanun RG: Resmî Gazete s. K.: sayılı Kanun TCK: Türk Ceza Kanunu TTK: Türk Ticaret Kanunu a İÇİNDEKİLER Bölüm 1 HUKUKUN BİLGİ KAYNAKLARI VE ATIF USÛLLERİ I. Bilgi Kaynaklan...................................................................................................................... 1 A. Mevzuat................................................................................... 1 B. Yargı Kararları..................................................................................................................

In [ ]:
!pip install jsonlines
!pip install jsonline

In [ ]:
import pandas as pd
import jsonlines  # dikkat: doğru modül ismi bu

# Dosya yolu
dosya_yolu = '/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl'

# Satırları listeye al
veri = []
with jsonlines.open(dosya_yolu) as reader:
    for satir in reader:
        veri.append(satir)

veri

[{'batch': 1,
  'title': 'Batch 1',
  'prompt': 'KISALTMALAR AY: Anayasa BK: Borçlar Kanunu CK: Ceza Kanunu CBK: Cumhurbaşkanlığı Kararnamesi CMUK: Ceza Muhakemeleri Usûlü Kanunu CMK: Ceza Muhakemesi Kanunu f.: Fıkra HUMK: Hukuk Usûlü Muhakemeleri Kanunu HMK: Hukuk Usûlü Muhakemeleri Kanunu İİK: İcra ve İflas Kanunu ÎYUK: İdarî Yargılama Usûlü Kanunu KHK: Kanun hükmünde kararname m.: Madde MK: Medeni Kanun RG: Resmî Gazete s. K.: sayılı Kanun TCK: Türk Ceza Kanunu TTK: Türk Ticaret Kanunu a İÇİNDEKİLER Bölüm 1 HUKUKUN BİLGİ KAYNAKLARI VE ATIF USÛLLERİ I. Bilgi Kaynaklan...................................................................................................................... 1 A. Mevzuat................................................................................... 1 B. Yargı Kararları..........................................................................................................................2 C. Bilimsel Eserler..............................................

In [ ]:
from datasets import load_dataset

hf_dataset = load_dataset("json", data_files='/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

model_name = "lightblue/DeepSeek-R1-Distill-Qwen-7B-Multilingual"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

from datasets import load_dataset

hf_dataset = load_dataset("json", data_files='/content/drive/MyDrive/Classroom/hukuk_finetune_temizlenmis (1).jsonl', split="train")

def tokenize_function(example):
    text = example["prompt"] + example["completion"]  # Prompt ve Completion birleştirildi
    return tokenizer(text, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = hf_dataset.map(tokenize_function)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./hukuk-ai",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()


config.json:   0%|          | 0.00/767 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,4.032900


TrainOutput(global_step=18, training_loss=4.077915615505642, metrics={'train_runtime': 153.8903, 'train_samples_per_second': 1.053, 'train_steps_per_second': 0.117, 'total_flos': 3302761345056768.0, 'train_loss': 4.077915615505642, 'epoch': 2.888888888888889})

In [ ]:
# Modeli eval moduna alıyoruz
model.eval()  # Modeli eval moduna al

def hukuk_ai_cevapla(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

soru = (
    "Soru: Türk Medeni Kanunu'na göre resmi evlilik için yasal yaş sınırı nedir?\n"
    "Lütfen yasal dayanağı ile birlikte açıklayıcı bir şekilde cevap ver.\nCevap:"
)
cevap = hukuk_ai_cevapla(soru)
print(cevap)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Soru: Türk Medeni Kanunu'na göre resmi evlilik için yasal yaş sınırı nedir?
Lütfen yasal dayanağı ile birlikte açıklayıcı bir şekilde cevap ver.
Cevap: Resmi evlilik için yasal yaş sınırı, **34 yaş**'dır. Bu yaşa göre, bir kişinin doğduktan 34 yıl sonra öldüğünde doğrulan "evlilik" kabul edilir. Bu yaşla doğrulan evlilik, medenime dayanak olarak kabul edilir. Ancak, bu yaşla doğrulan evlilikte, beş ay içinde doğan nesillerin de evlilik kabul edilmez.

**Yasal Dayanak**:
- **Evlilik Yarısın**: Bir kişinin doğduktan 34 yıl sonra doğan kişiye evlilik kabul edilir.
- **Yasal İstikrarı**: 34 yaşa göre doğan nesillerin evlilik kabul edilmez.
- **Yasal Dayanak**: Evlilik kabul
